<img src="header.png"></img>

<div align="center">
    <h1>Report on Participation in Kaggle Competition</h1>
    <h1>Part II: Feature Engineering</h1>
</div>

<div align="center"><i>
In this Jupyter notebook I am engineering features from 4 data sheets provided by competition originator.<br/>
<br/>
Prepared by Artem Drofa.
</i></div>

<a id='main_takeaways'></a>
## Main Takeaways
1. Categorical data in train and test data sheets have been one-hot encoded.
2. Historical Transactions have been splitted into authorized and denied (no denied among New Merchant transactions), finally identical feature have been engineered out of 3 data sheets:
   * historical authorized transactions;
   * historical denied transactions;
   * new merchant transactions.
3. In mentioned transactions data sheets:
   * numerical columns have been aggregated by card_id and following values have been estimated: sum, mean, min, max and var values;
   * categorical columns have been one-hot encoded, then aggregated by card_id and following values have been estimated: sum and sum devided on amount of transactions performed by card_id (i.e. share of transactions in each category). <font color='blue'><i>(Due to limited capacity of my laptop I had to perform this operation using sparse matrix multiplication (not pandas aggregation), may be, this 'trick' will be interesting to dear reader)</i></font>;
   * results have been merged with train and test data sheets (key = card_id).
4. Cosine distance between historical and new merchant transactions has been added for trying to identify influence of Elo's recomendation algorithms on loyalty score.
5. Data sheets prepared for model training:
   * train_df.pkl;
   * test_df.pkl;
   * target_df.pkl.

## Feature Engineering

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from scipy.sparse import csr_matrix, hstack
from scipy.spatial import distance

import os
import pickle
import copy
from IPython.core import display as ICD

import gc
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore')

In [2]:
PATH_TO_DATA = '.../data'

In [3]:
# Reduce Memory Function
# (https://www.kaggle.com/roydatascience/elo-stack-interactions-on-categorical-variables)
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
                      .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### Train and Test Data Sheets

In [4]:
with open(os.path.join(PATH_TO_DATA, 'train.pkl'), 'rb') as pkl_f:
    train = pickle.load(pkl_f)
with open(os.path.join(PATH_TO_DATA, 'test.pkl'), 'rb') as pkl_f:
    test = pickle.load(pkl_f)

Train and test data sheets preview:

In [5]:
ICD.display(train.head())
ICD.display(test.head())

,first_active_month,card_id,feature_1,feature_2,feature_3,target
card_id,,,,,,
C_ID_92a2005557,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
C_ID_3d0044924f,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913
C_ID_d639edf6cd,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056
C_ID_186d6a6901,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495
C_ID_cdbd2c0db2,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749


,first_active_month,card_id,feature_1,feature_2,feature_3
card_id,,,,,
C_ID_0ab67a22ab,2017-04-01,C_ID_0ab67a22ab,3,3,1
C_ID_130fd0cbdd,2017-01-01,C_ID_130fd0cbdd,2,3,0
C_ID_b709037bc5,2017-08-01,C_ID_b709037bc5,5,1,1
C_ID_d27d835a9f,2017-12-01,C_ID_d27d835a9f,2,1,0
C_ID_2b5e3df5c2,2015-12-01,C_ID_2b5e3df5c2,5,1,1


In [6]:
train_test = train.append(test, sort=False)
del train, test
gc.collect()
pass

#### first_active_month

In [7]:
print('# of unique first_active_month:', train_test['first_active_month'].unique().shape[0])

# of unique first_active_month: 77


I'd like to represent first_active_month as a difference in days from some day in the future (fut_day). Let' take max(first_active_date, purchase_date)+1 as fut_day.

In [8]:
#open hist_ and new_ -trans data sheets
with open(os.path.join(PATH_TO_DATA, 'hist_trans.pkl'), 'rb') as pkl_f:
    hist_trans = pickle.load(pkl_f)
with open(os.path.join(PATH_TO_DATA, 'new_trans.pkl'), 'rb') as pkl_f:
    new_trans = pickle.load(pkl_f)

#hist_ and new_ -trans (purchase) dates
htd = hist_trans['purchase_date']
ntd = new_trans['purchase_date']

#hist_ and new_ -trans data sheets utilize a lot memory, let's clear it for now
del hist_trans, new_trans
gc.collect()

#creating day in the future
fut_day = max(train_test['first_active_month'].max(), htd.max(), ntd.max()) + pd.DateOffset(1)
fut_day = pd.to_datetime(fut_day.strftime('%Y-%m-%d'))

#adding to train_test
train_test['first_active_month_age'] = (fut_day - train_test['first_active_month']).dt.days

#### feature_1, feature_2, feature_3

In [9]:
print('Unique values:')
for f in ['feature_1', 'feature_2', 'feature_3']:
    print('-', f, train_test[f].unique())

Unique values:
- feature_1 [5 4 2 1 3]
- feature_2 [2 1 3]
- feature_3 [1 0]


feature_1 and feature_2 are ctegorical (anonymized), number of unique values is above 2, thus I'll one-hot encode this features.

In [10]:
for f in ['feature_1', 'feature_2']:
    dummies = pd.get_dummies(train_test[f], prefix=f, drop_first=True)
    train_test = train_test.merge(dummies, how='left', left_index=True, right_index=True)

In [11]:
train_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_month_age,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_2,feature_2_3
card_id,,,,,,,,,,,,,
C_ID_92a2005557,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,334,0,0,0,1,1,0
C_ID_3d0044924f,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,485,0,0,1,0,0,0
C_ID_d639edf6cd,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,638,1,0,0,0,1,0
C_ID_186d6a6901,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,242,0,0,1,0,0,1
C_ID_cdbd2c0db2,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,181,0,0,0,0,0,1


Saving and deleting train_test data sheet for memory clearing.

In [12]:
with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
    pickle.dump(train_test, pkl_f)

del train_test
gc.collect()
pass

### Historical and New Merchant Transactions Data Sheets
1. Non-categorical columns;
2. Categorical columns;
3. Cosine distance between hist and new merchant transaction to reflect activation of Elo's recommendation system.

In [13]:
with open(os.path.join(PATH_TO_DATA, 'hist_trans.pkl'), 'rb') as pkl_f:
    hist_trans = pickle.load(pkl_f)
with open(os.path.join(PATH_TO_DATA, 'new_trans.pkl'), 'rb') as pkl_f:
    new_trans = pickle.load(pkl_f)

First of all, I split hist_trans data sheet into (1) hist_trans with authorized transactions and (2) hist_trans with denied transactions.<br/>
<i>New_trans data sheet doesn't include unauthorized transactions.</i>

In [14]:
hist_trans_auth = hist_trans[hist_trans['authorized_flag'] == 1]
hist_trans_den = hist_trans[hist_trans['authorized_flag'] == 0]

#indexing
for ds in [hist_trans_auth, hist_trans_den, new_trans]:
    ds.index = ds['card_id']

#memory clearing
del hist_trans
gc.collect()
pass

#### Non-Categorical Columns

There are following non-categorical columns in hist_ and new_ -trans data sheets:
* installments;
* month_lag;
* purchase_amount;
* purchase_date;
* ref_date_1MY (quasi-linear combination of purchase_date and month_lag).

Basing on this columns I'll create additional columns:
* purchase_amount / installments;
* purchase_date_age (difference between a date from future and purchase_date);
* ref_date_1MY_age (difference between a date from future and purchase_date).

After this purchase_date and ref_date_1MY could be dropped. All non-categorical columns will be aggregated by crad_id resulting in sum, min, max, mean and variance values.

In [15]:
#creating additional columns
for ds in [hist_trans_auth, hist_trans_den, new_trans]:
    
    ds['purchase_amount_per_installments'] =\
    ds['purchase_amount'] / ds['installments'].replace([0, 999], np.nan)
    
    ds['purchase_date_age'] = (fut_day - ds['purchase_date']).dt.total_seconds()
    ds['ref_date_1MY_age'] = (fut_day - ds['ref_date_1MY']).dt.total_seconds()
    
    ds = ds.drop(columns=['purchase_date', 'ref_date_1MY'])

In [16]:
#columns to aggregate
non_cat_col = ['installments', 'month_lag', 'purchase_amount',
               'purchase_date_age', 'ref_date_1MY_age', 'purchase_amount_per_installments']

#aggregation parameters
aggs = {}
for col in non_cat_col:
    aggs[col] = ['sum', 'max', 'min', 'mean', 'var']

In [17]:
def train_test_non_cat():
    #open train_test for merging
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)

    for ds, ds_name in tqdm(zip([hist_trans_auth, hist_trans_den, new_trans], ['ht_auth', 'ht_den', 'nt'])):
        #aggregating
        ds_agg = ds.reset_index(drop=True).groupby('card_id').agg(aggs)

        #renaming columns
        ds_agg.columns = [ds_name+'_'+i+'_'+j for i, j in zip(ds_agg.columns.get_level_values(0),
                                                              ds_agg.columns.get_level_values(1))]
        #merging
        train_test = train_test.merge(ds_agg, how='left', left_index=True, right_index=True)

        #memory clearing
        del ds_agg
        gc.collect()

    #saving train_test and clearing memory
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
        pickle.dump(train_test, pkl_f)
    del train_test
    gc.collect()
    pass

In [18]:
%%time
train_test_non_cat()

3it [00:50, 23.31s/it]


CPU times: user 21.6 s, sys: 11.6 s, total: 33.2 s
Wall time: 51.7 s


#### Categorical Columns

Now let's have a look at categorical columns in hist_ and new_ -trans data sheets

In [19]:
cat_cols = ['city_id', 'category_1', 'category_3', 'merchant_category_id',
           'merchant_id', 'category_2', 'state_id', 'subsector_id']

In [20]:
for col in cat_cols:
    values = hist_trans_auth[col]
    for ds in [hist_trans_den, new_trans]:
        values.append(ds[col])
    print(col, ':', values.unique().shape[0])

city_id : 308
category_1 : 2
category_3 : 4
merchant_category_id : 327
merchant_id : 324323
category_2 : 6
state_id : 25
subsector_id : 41


* All columns beside category_1 and merchant_id will be one-hot encoded. After I will aggregate columns by card_id and estimate sum and mean of each column: sum will show total amount of transactions in each category and mean will show share of transactions in each category.
* Category_1 has only 2 unique values, it should be binarized, mean and sum should be aggregated.
* Merchant_id has to much values for one-hot encoding, hence only number of unique values for each card_id will be added to train_test;

##### One-Hot Encoding and Aggreation

Unfortunately my laptop is not able to perform one-hot encoding and aggregate sums and means using pandas, hence <b>I transform data using scipy OneHotEncoder and aggregate it using sparse matrix multiplication</b>.

In [21]:
ohe_cols = ['city_id', 'category_3', 'merchant_category_id', 'category_2', 'state_id', 'subsector_id']

In [22]:
%%time
#transactions (files) dump
for ds, ds_name in tqdm(zip([hist_trans_auth, hist_trans_den, new_trans], ['ht_auth', 'ht_den', 'nt'])):
    with open(os.path.join(PATH_TO_DATA, ds_name+'.pkl'), 'wb') as pkl_f:
        pickle.dump(ds, pkl_f)

#clearing memory
del hist_trans_auth, hist_trans_den, new_trans
gc.collect()
pass

3it [00:57, 26.85s/it]


CPU times: user 11.9 s, sys: 7.46 s, total: 19.4 s
Wall time: 59.3 s


In the following cell I:
* perform one-hot encoding for each of selected feature in each of 3 data sheets (historical transaction authorized, historical transactions denied, new merchant transasctions;
* agregate received columns by card_id and estimate sum and mean using sparse matrix multiplication;
* save received matrices.

After I will stack all matrices for each data sheet and merge it by card_id with train_test data sheet.

In [23]:
def ohe_agg():
    for ds_name in ['ht_auth', 'ht_den', 'nt']:
        #open data sheet
        with open(os.path.join(PATH_TO_DATA, ds_name+'.pkl'), 'rb') as pkl_f:
            ds = pickle.load(pkl_f)

        #STEP I: preparation of card_id matrix for further multiplication for aggregation purposes 
        card_id = ds['card_id']
        card_id_enc = OneHotEncoder()
        card_id_ohe = card_id_enc.fit_transform(card_id.values.reshape(-1, 1))

        vector_slicer = np.vectorize(lambda a: a[3:])
        card_id_index = vector_slicer(card_id_enc.get_feature_names())
        
        #STEP II: saving card_id index
        with open(os.path.join(PATH_TO_DATA, ds_name+'_card_id.pkl'), 'wb') as pkl_f:
                pickle.dump(card_id_index, pkl_f)

        #memory clearing
        del card_id, card_id_enc
        del vector_slicer, card_id_index
        gc.collect()

        #STEP III: columns encoding
        for col in tqdm(ohe_cols):
            col_enc = OneHotEncoder()
            col_ohe = col_enc.fit_transform(ds[col].astype(str).fillna('NaN').values.reshape(-1, 1))
            col_names = col_enc.get_feature_names(input_features=[col])

            #STEP IV: aggregation
            col_sum = (col_ohe.T @ card_id_ohe).T #SUM
            col_mean = csr_matrix(col_sum / card_id_ohe.sum(axis=0).T) #MEAN (i.e. share)

            #STEP VI: Aggregating sparse matrices and columns names
            col_agg = hstack([col_sum, col_mean]) #index=card_id_index,
            col_names_agg = [ds_name+'_'+col_name+'_sum' for col_name in col_names] +\
                            [ds_name+'_'+col_name+'_mean' for col_name in col_names]


            #STEP VII: saving transformed features and its names
            with open(os.path.join(PATH_TO_DATA, ds_name+'_'+col+'_ohe_features.pkl'), 'wb') as pkl_f:
                pickle.dump(col_agg, pkl_f)
            with open(os.path.join(PATH_TO_DATA, ds_name+'_'+col+'_ohe_features_names.pkl'), 'wb') as pkl_f:
                pickle.dump(col_names_agg, pkl_f)

            #memory clearing
            del col_enc, col_ohe, col_names
            del col_sum, col_mean
            del col_agg, col_names_agg
            gc.collect()
        
        #memory clearing
        del card_id_ohe
        gc.collect()

In [24]:
%%time
ohe_agg()

100%|██████████| 6/6 [00:13<00:00,  2.18s/it]

CPU times: user 2min 32s, sys: 34.1 s, total: 3min 7s
Wall time: 3min 8s


In the cell below I h-stack all received matrices into 1 DataFrame.

In [25]:
def ohe_hstack():
    for ds_name in ['ht_auth', 'ht_den', 'nt']:

        #STEP I: initializing matrix with features and column names for it with 1st feature
        with open(os.path.join(PATH_TO_DATA, ds_name+'_'+ohe_cols[0]+'_ohe_features.pkl'), 'rb') as pkl_f:
                col_agg_full = pickle.load(pkl_f)
        with open(os.path.join(PATH_TO_DATA, ds_name+'_'+ohe_cols[0]+'_ohe_features_names.pkl'), 'rb') as pkl_f:
                col_names_full = pickle.load(pkl_f)

        #STEP II: h-stacking all matrices and column names
        for col in tqdm(ohe_cols[1:]):
            with open(os.path.join(PATH_TO_DATA, ds_name+'_'+col+'_ohe_features.pkl'), 'rb') as pkl_f:
                col_agg = pickle.load(pkl_f)
            with open(os.path.join(PATH_TO_DATA, ds_name+'_'+col+'_ohe_features_names.pkl'), 'rb') as pkl_f:
                col_names = pickle.load(pkl_f)

            col_agg_full = hstack([col_agg_full, col_agg])
            col_names_full += col_names

            #memory clearing
            del col_agg, col_names
            gc.collect()

        #STEP III: open card_id index
        with open(os.path.join(PATH_TO_DATA, ds_name+'_card_id.pkl'), 'rb') as pkl_f:
            card_id = pickle.load(pkl_f)

        #STEP IV: DataFrame creation and saving
        df = pd.DataFrame(col_agg_full.todense(), index=card_id, columns=col_names_full)
        with open(os.path.join(PATH_TO_DATA, ds_name+'_ohe_df.pkl'), 'wb') as pkl_f:
                pickle.dump(df, pkl_f)

        #memory clearing
        del df
        gc.collect()

In [26]:
%%time
ohe_hstack()

100%|██████████| 5/5 [00:01<00:00,  3.69it/s]


CPU times: user 14 s, sys: 20.7 s, total: 34.6 s
Wall time: 1min 33s


Deleting support files.

In [27]:
def del_sup_files():
    for ds_name in tqdm(['ht_auth', 'ht_den', 'nt']):

        for col in ohe_cols:
            os.remove(os.path.join(PATH_TO_DATA, ds_name+'_'+col+'_ohe_features.pkl'))
            os.remove(os.path.join(PATH_TO_DATA, ds_name+'_'+col+'_ohe_features_names.pkl'))
        
        os.remove(os.path.join(PATH_TO_DATA, ds_name+'_card_id.pkl'))

    gc.collect()
    pass

In [28]:
%%time
del_sup_files()

100%|██████████| 3/3 [00:00<00:00, 309.39it/s]

CPU times: user 48.4 ms, sys: 0 ns, total: 48.4 ms
Wall time: 54.4 ms


Reducing memory usage of created DataFrames

In [29]:
def reduce_mem_ohe_df():
    for ds_name in ['ht_auth', 'ht_den', 'nt']:

        #open df
        with open(os.path.join(PATH_TO_DATA, ds_name+'_ohe_df.pkl'), 'rb') as pkl_f:
            df = pickle.load(pkl_f)

        #reduce memory usage
        df = reduce_mem_usage(df)

        #save df
        with open(os.path.join(PATH_TO_DATA, ds_name+'_ohe_df.pkl'), 'wb') as pkl_f:
            pickle.dump(df, pkl_f)

        #memory clearing
        del df
        gc.collect()
        pass

In [30]:
%%time
reduce_mem_ohe_df()

100%|██████████| 1422/1422 [14:19<00:00,  1.65it/s]


Mem. usage decreased to 885.43 Mb (74.9% reduction)


100%|██████████| 1390/1390 [11:10<00:00,  2.07it/s]


Mem. usage decreased to 733.05 Mb (74.9% reduction)


100%|██████████| 1396/1396 [11:47<00:00,  1.97it/s]


Mem. usage decreased to 774.39 Mb (74.9% reduction)
CPU times: user 22min 5s, sys: 51min 40s, total: 1h 13min 45s
Wall time: 38min 3s


In the cell below I merge all of created DataFrames with train_test data sheet.

In [31]:
def df_train_test_merge():
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)

    for ds_name in tqdm(['ht_auth', 'ht_den', 'nt']):

        #open df
        with open(os.path.join(PATH_TO_DATA, ds_name+'_ohe_df.pkl'), 'rb') as pkl_f:
            df = pickle.load(pkl_f)

        #merge train_test with df
        train_test = train_test.merge(df, how='left', left_index=True, right_index=True)

        #memory clearing
        del df
        gc.collect()

    #saving train_test and clearing memory
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
        pickle.dump(train_test, pkl_f)
    del train_test
    gc.collect()
    pass

In [32]:
%%time
df_train_test_merge()

100%|██████████| 3/3 [00:26<00:00,  8.57s/it]


CPU times: user 16.3 s, sys: 8.06 s, total: 24.3 s
Wall time: 1min 3s


Reducing train_test memory usage.

In [33]:
def reduce_mem_train_test():
    #open train_test
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)

    #reduce memory usage
    train_test = reduce_mem_usage(train_test)

    #save train_test
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
        pickle.dump(train_test, pkl_f)

    #memory clearing
    del train_test
    gc.collect()
    pass

In [34]:
%%time
reduce_mem_train_test()

100%|██████████| 4311/4311 [00:49<00:00, 87.19it/s] 


Mem. usage decreased to 2699.44 Mb (5.4% reduction)
CPU times: user 52.8 s, sys: 4.71 s, total: 57.5 s
Wall time: 1min 12s


##### category_1 and merchant_id

In [35]:
def category_1_merchant_id():
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)

    aggs_2 = {'category_1':['sum', 'mean'],
            'merchant_id':['nunique']}

    for ds_name in tqdm(['ht_auth', 'ht_den', 'nt']):

        #open data sheet
        with open(os.path.join(PATH_TO_DATA, ds_name+'.pkl'), 'rb') as pkl_f:
            ds = pickle.load(pkl_f)

        #binarization of category_1
        ds['category_1'] = ds['category_1'].map({'Y':1, 'N':0})

        #aggregating
        ds_agg = ds.reset_index(drop=True).groupby('card_id').agg(aggs_2)

        #renaming columns
        ds_agg.columns = [ds_name+'_'+i+'_'+j for i, j in zip(ds_agg.columns.get_level_values(0),
                                                              ds_agg.columns.get_level_values(1))]

        #merging
        train_test = train_test.merge(ds_agg, how='left', left_index=True, right_index=True)

        #clearing memory
        del ds, ds_agg
        gc.collect()

    #saving train_test and clearing memory
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
        pickle.dump(train_test, pkl_f)
    del train_test
    gc.collect()
    pass

In [36]:
%%time
category_1_merchant_id()

100%|██████████| 3/3 [02:59<00:00, 85.53s/it] 


CPU times: user 48.9 s, sys: 33 s, total: 1min 21s
Wall time: 3min 19s


#### Cosine Distance

I would like to remind that historical and new merchant transactions are chronologically splitted by some event, supposingly, this event is a launch of Elo's merchant reccomendation algorithm. New merchants transaction data sheet includes information only about transations in merchants which have been never met before for each card_id.

The idea, which lies on a surface, is to recomend to a client new merchants from merchant categories in which this client has most of his transactions (by quantity or sum).

Basing on these I'd like to create 2 additional features for each card_id:
1. cosine distance between vectors of <u>quantity</u> of purchases aggregated by merchant_category_id from historical authorized and new merchant transactions;
2. cosine distance between vectors of <u>sum</u> of purchases aggregated by merchant_category_id from historical authorized and new merchant transactions.

##### Quantity of Transactions

In [37]:
def cos_dist_quantity():

    #opening transactions files
    with open(os.path.join(PATH_TO_DATA, 'ht_auth.pkl'), 'rb') as pkl_f:
        ht_auth = pickle.load(pkl_f)
    with open(os.path.join(PATH_TO_DATA, 'nt.pkl'), 'rb') as pkl_f:
        nt = pickle.load(pkl_f)
    print('files opened')

    #generating vectors
    ht_vec = ht_auth.reset_index(drop=True).groupby(['card_id', 'merchant_category_id']
                                                   )['merchant_category_id'].size().unstack(fill_value=0)
    nt_vec = nt.reset_index(drop=True).groupby(['card_id', 'merchant_category_id']
                                              )['merchant_category_id'].size().unstack(fill_value=0)
    print('vectors generated')

    #clearing memory
    del ht_auth, nt
    gc.collect()

    #finding out missing merchant categories in vecs
    ht_miss_cat = np.setdiff1d(nt_vec.columns, ht_vec.columns)
    nt_miss_cat = np.setdiff1d(ht_vec.columns, nt_vec.columns)
    print('missing categories found')

    #fullfilling missing merchant categories
    for cat in ht_miss_cat:
        ht_vec[cat] = 0
    for cat in nt_miss_cat:
        nt_vec[cat] = 0
    print('missing categories fullfilled')

    #sorting columns
    cols_sorted = np.sort(ht_vec.columns)
    print('columns sorted')

    ht_vec = ht_vec[cols_sorted]
    nt_vec = nt_vec[cols_sorted]

    #renaming columns
    ht_vec.columns = ['ht_'+str(col) for col in cols_sorted]
    nt_vec.columns = ['nt_'+str(col) for col in cols_sorted]
    print('columns renamed')

    #merging vecs and fill NaNs with 0
    ht_nt_vec = pd.concat([ht_vec, nt_vec], axis=1, sort=False).fillna(0)
    print('vecs merged, NaNs filled')

    #estimating cosine distance
    dot_product = np.einsum('ij,ij->i', ht_nt_vec[ht_vec.columns], ht_nt_vec[nt_vec.columns])
    ht_vec_mod = (ht_nt_vec[ht_vec.columns] ** 2).sum(axis=1) ** (1/2)
    nt_vec_mod = (ht_nt_vec[nt_vec.columns] ** 2).sum(axis=1) ** (1/2)
    ht_nt_vec['cos_dist_count_by_merchant_category'] = 1 - dot_product / (ht_vec_mod * nt_vec_mod)
    print('cosine distance estimated')

    #adding created feature to train_test
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)
    print('train_test opened')

    train_test = train_test.merge(ht_nt_vec[['cos_dist_count_by_merchant_category']],
                                  how='left', left_index=True, right_index=True)
    print('features added')

    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
        pickle.dump(train_test, pkl_f)
    print('train_test dumped')

    #memory clearing
    del train_test, ht_vec, nt_vec, ht_nt_vec
    gc.collect()
    pass

In [38]:
%%time
cos_dist_quantity()

files opened
vectors generated
missing categories found
missing categories fullfilled
columns sorted
columns renamed
vecs merged, NaNs filled
cosine distance estimated
train_test opened
features added
train_test dumped
CPU times: user 38.2 s, sys: 26.9 s, total: 1min 5s
Wall time: 2min 38s


##### Quantity of Transactions

In [39]:
def cos_dist_purchase_amount():
    
    #opening transactions files
    with open(os.path.join(PATH_TO_DATA, 'ht_auth.pkl'), 'rb') as pkl_f:
        ht_auth = pickle.load(pkl_f)
    with open(os.path.join(PATH_TO_DATA, 'nt.pkl'), 'rb') as pkl_f:
        nt = pickle.load(pkl_f)
    print('files opened')
    
    #generating vectors
    ht_vec = ht_auth.reset_index(drop=True).groupby(['card_id', 'merchant_category_id']
                                                   )['purchase_amount'].sum().unstack(fill_value=0)
    nt_vec = nt.reset_index(drop=True).groupby(['card_id', 'merchant_category_id']
                                              )['purchase_amount'].sum().unstack(fill_value=0)
    print('vectors generated')
    
    #clearing memory
    del ht_auth, nt
    gc.collect()

    #finding out missing merchant categories in vecs
    ht_miss_cat = np.setdiff1d(nt_vec.columns, ht_vec.columns)
    nt_miss_cat = np.setdiff1d(ht_vec.columns, nt_vec.columns)
    print('missing categories found')

    #fullfilling missing merchant categories
    for cat in ht_miss_cat:
        ht_vec[cat] = 0
    for cat in nt_miss_cat:
        nt_vec[cat] = 0
    print('missing categories fullfilled')

    #sorting columns
    cols_sorted = np.sort(ht_vec.columns)
    print('columns sorted')

    ht_vec = ht_vec[cols_sorted]
    nt_vec = nt_vec[cols_sorted]

    #renaming columns
    ht_vec.columns = ['ht_'+str(col) for col in cols_sorted]
    nt_vec.columns = ['nt_'+str(col) for col in cols_sorted]
    print('columns renamed')

    #merging vecs and fill NaNs with 0
    ht_nt_vec = pd.concat([ht_vec, nt_vec], axis=1, sort=False).fillna(0)
    print('vecs merged, NaNs filled')
    
    #estimating cosine distance
    dot_product = np.einsum('ij,ij->i', ht_nt_vec[ht_vec.columns], ht_nt_vec[nt_vec.columns])
    ht_vec_mod = (ht_nt_vec[ht_vec.columns] ** 2).sum(axis=1) ** (1/2)
    nt_vec_mod = (ht_nt_vec[nt_vec.columns] ** 2).sum(axis=1) ** (1/2)
    ht_nt_vec['cos_dist_sum_by_merchant_category'] = 1 - dot_product / (ht_vec_mod * nt_vec_mod)
    print('cosine distance estimated')
    
    #adding created feature to train_test
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)
    print('train_test opened')

    train_test = train_test.merge(ht_nt_vec[['cos_dist_sum_by_merchant_category']],
                                  how='left', left_index=True, right_index=True)
    print('features added')

    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'wb') as pkl_f:
        pickle.dump(train_test, pkl_f)
    print('train_test dumped')

    #memory clearing
    del train_test, ht_vec, nt_vec, ht_nt_vec
    gc.collect()
    pass

In [40]:
%%time
cos_dist_purchase_amount()

files opened
vectors generated
missing categories found
missing categories fullfilled
columns sorted
columns renamed
vecs merged, NaNs filled
cosine distance estimated
train_test opened
features added
train_test dumped
CPU times: user 47.9 s, sys: 58 s, total: 1min 45s
Wall time: 4min 58s


### Saving prepared <u>train_df</u>, <u>test_df</u> and <u>target_df</u> data sheets

In [41]:
def df_saving():
    with open(os.path.join(PATH_TO_DATA, 'train_test.pkl'), 'rb') as pkl_f:
        train_test = pickle.load(pkl_f)

    train = train_test[train_test['target'].notnull()]
    test = train_test[train_test['target'].isnull()]

    #clearing memory
    del train_test
    gc.collect()
    pass

    target = train['target']
    
    feats_to_drop = ['card_id', 'target', 'first_active_month', 'feature_1', 'feature_2']
    train = train.drop(columns=feats_to_drop)
    test = test.drop(columns=feats_to_drop)
    
    for df, df_name in zip([train, test, target], ['train', 'test', 'target']):
        with open(os.path.join(PATH_TO_DATA, df_name+'_df.pkl'), 'wb') as pkl_f:
            pickle.dump(df, pkl_f)

    #clearing memory
    del train, test, target
    gc.collect()
    pass

In [42]:
%%time
df_saving()

CPU times: user 12.4 s, sys: 42.8 s, total: 55.2 s
Wall time: 4min 36s


## Conclusion
Please, consider ['Main Takeaways'](#main_takeaways) at the top of the page.